In [29]:
import pandas as pd
import numpy as np
import sys
from scipy.sparse import *
from scipy import *
import pickle

In [38]:
# read training set
training = pd.read_csv("data/training_data.csv")[['user', 'track']]

#training = training.iloc[:10000]
training.head()


,user,track
0,000c556aa021426138c47076d53dfbec,spotify:track:741YBRPcuQo58RkVygeAey
1,000c556aa021426138c47076d53dfbec,spotify:track:4e2jNdR9J0PoubH0CqQm4j
2,000c556aa021426138c47076d53dfbec,spotify:track:467KUJ5Gm4p8znEpVKK5i5
3,000c556aa021426138c47076d53dfbec,spotify:track:2JO3HwMRPeya8bXbtbyPcf
4,000c556aa021426138c47076d53dfbec,spotify:track:7kw6yVmdZGuRcWGi4adXYT


In [39]:
tracks = training['track'].unique()
tracks_size = len(tracks)

users = training['user'].unique()
users_size = len(users)


#items = np.zeros((size,size))
users_sparse = csr_matrix( (users_size,tracks_size), dtype=int16 )


print(users_sparse.shape)
      


track_id_dict = {}
id_track_dict = {}
for i,track in enumerate(tracks):
    track_id_dict[track] = i
    id_track_dict[i] = track
    

user_id_dict = {}
id_user_dict = {}
for i,user in enumerate(users):
    user_id_dict[user] = i
    id_user_dict[i] = user
    
    
with open('cache/track_to_id.pickle', 'wb') as handle:
    pickle.dump(track_id_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)    
with open('cache/id_to_track.pickle', 'wb') as handle:
    pickle.dump(id_track_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)     
    
with open('cache/user_to_id.pickle', 'wb') as handle:
    pickle.dump(user_id_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)    
with open('cache/id_to_user.pickle', 'wb') as handle:
    pickle.dump(id_user_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

(569, 476161)


In [41]:
# Create user dict
user_dict = {}
groups = training.groupby('user')
for i, group in groups:
    user_id = group['user'].iloc[0]
    track_ids = set(group['track'])
    user_dict[user_id] = track_ids

In [42]:
# Generate user item matrix
count = 0
for user, tracks in user_dict.items():
    track_ids = [track_id_dict[track] for track in tracks]
    user_id = user_id_dict[user]
    sys.stdout.write("User: "+str(count)+"\r")
    sys.stdout.flush() 
    count += 1
    #if len(track_ids) < 1500 :
    for track_id in track_ids:
        #print(track_id_dict[track_id])
        row = array([user_id for i in track_ids])
        col = array(track_ids)
        data = array([1 for i in track_ids])
        acc = csr_matrix( (data,(row,col)), shape=(users_size,tracks_size), dtype=int16 )
        users_sparse += acc
        
        

In [17]:
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=150, algorithm='arpack')
svd.fit(users_sparse)
users_reduced = svd.transform(users_sparse)

In [18]:
save_npz('cache/users_sparse.npz', users_sparse)
np.save('cache/users_reduced.npy', users_reduced)

with open('cache/users_svd.pickle', 'wb') as handle:
    pickle.dump(svd, handle, protocol=pickle.HIGHEST_PROTOCOL)  

# Test

In [19]:
import pandas as pd
training = pd.read_csv("data/training_data.csv")[['user', 'track']]
test = pd.read_csv("data/test_data.csv")[['user', 'track']]
test.head()

,user,track
0,000c556aa021426138c47076d53dfbec,spotify:track:542sbDYHLJ7fVwwbSWG9BD
1,000c556aa021426138c47076d53dfbec,spotify:track:6fgjU6IfBOXHI3OKtndEeE
2,000c556aa021426138c47076d53dfbec,spotify:track:6t8NCBZWMOxj2Odg9gIBOF
3,000c556aa021426138c47076d53dfbec,spotify:track:6r9WmN9czMIbpRErhbOTKh
4,000c556aa021426138c47076d53dfbec,spotify:track:3cTblov6gyVDgvk5cTMFQA


In [20]:
user_dict_train = {}
groups = training.groupby('user')
for i, group in groups:
    user_id = group['user'].iloc[0]
    track_ids = set(group['track'])
    user_dict_train[user_id] = track_ids

In [21]:
user_dict_test = {}
groups = test.groupby('user')
for i, group in groups:
    user_id = group['user'].iloc[0]
    track_ids = set(group['track'])
    user_dict_test[user_id] = track_ids

In [32]:
tracks_train = user_dict_train['000c556aa021426138c47076d53dfbec']

In [33]:
tracks_test = user_dict_test['000c556aa021426138c47076d53dfbec']

In [34]:
import recommender_cf_user_based as ubrec

In [35]:
tracks_result = ubrec.recommend(tracks_train, 500)
#tracks_result

[[  1.00000000e+00   8.72200545e-02   0.00000000e+00   9.89159034e-03
    8.12233791e-03   1.73233848e-02   0.00000000e+00   2.59887943e-02
    5.69751810e-03   2.69625479e-03   0.00000000e+00   1.66299561e-02
    2.54097704e-03   0.00000000e+00   0.00000000e+00   1.45437646e-01
   -1.22678798e-05   2.32942796e-03   0.00000000e+00   0.00000000e+00
    5.99906578e-02   0.00000000e+00   0.00000000e+00   1.24835292e-02
    0.00000000e+00  -4.66970923e-03   4.23740307e-03  -6.74575570e-04
    2.86799548e-04   9.69562089e-04  -6.07953766e-04   4.68310974e-03
    2.92236611e-03   5.96495562e-04   4.37577762e-02   3.68800075e-05
    2.68889997e-02   1.25249787e-01   1.63518617e-02   2.57487095e-03
   -2.02956139e-03  -6.28418335e-03   7.66865663e-03   8.22695333e-04
    2.71903591e-02   5.30920324e-03   8.84600044e-03   0.00000000e+00
    1.37392303e-02   1.26440450e-01  -1.77138899e-03   2.89507178e-02
    7.74765903e-03   7.30009398e-02  -3.22013393e-03   0.00000000e+00
    8.80654980e-04  

In [37]:
def get_precision(result_query, result_given):
    if len(result_query) == 0:
        return 0
    else:
        return len(set(result_given).intersection(result_query))/len(result_query)
def get_recall(result_query, result_given):
    if len(result_given) == 0:
        return 0
    else:
        return len(set(result_given).intersection(result_query))/len(result_given)


precision = get_precision(tracks_result, tracks_test)
recall = get_recall(tracks_result, tracks_test)
    
#precision = get_precision([1, 2], [1, 2, 3, 4])
#recall = get_recall([1, 2], [1, 2, 3, 4])
    
print(precision)
print(recall)

0.0
0.0
